# Load and compile all snowlines and AOIs, plot results

In [ ]:
# !pip install contextily

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely import wkt
import seaborn as sns
import contextily as ctx
from tqdm.auto import tqdm
from ast import literal_eval

In [ ]:
# If using Google Colab, mount Google Drive so you can access the files in this folder
# from google.colab import drive
# drive.mount('/content/drive')

## Determine settings and grab site names from file

In [ ]:
# path to study-sites/
study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'

# determine whether to save figures to file
save_figures = True

# path to save output figures
figures_out_path = study_sites_path + '../figures/'

# Grab list of all study site names in folder
os.chdir(study_sites_path)
site_names = sorted([x[0:-1] for x in glob.glob('*/', recursive = True)])

# Filter sites for those with snowline files
site_names = sorted([x for x in site_names if len(glob.glob(study_sites_path + x + '/imagery/snowlines/*.csv')) > 0])
print('Number of sites = ' + str(len(site_names)))
site_names

## Load pre-saved AOIs and snowlines, check for new sites

In [ ]:
all_aois = pd.read_csv('all_aois.csv')
all_aois[['O1Region', 'O2Region']] = all_aois[['O1Region', 'O2Region']].astype(str).astype(float)
all_aois['geometry'] = all_aois['geometry'].apply(wkt.loads)
all_snowlines = pd.read_csv('all_snowlines.csv')
all_snowlines['datetime'] = all_snowlines['datetime'].astype('datetime64[ns]')
# determine which study sites do not have snowlines saved to file
bgotus_site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry'] # BGOTUS site names were changed in file
new_site_names = [x for x in site_names if (x not in set(all_snowlines['site_name'].values)) and (x not in bgotus_site_names)]
new_site_names

## Load and save snowlines and AOIs for new sites

In [ ]:
# If new sites were found, continue
if len(new_site_names) > 0:
    
    # iterate over new site names
    for site_name in new_site_names:

        print(site_name)

        # load AOI
        if (site_name=='Wolverine') or (site_name=='Gulkana') or (site_name=='LemonCreek') or (site_name=='SouthCascade') or (site_name=='LemonCreek'):
            aoi_fn = glob.glob(site_name + '/AOIs/*USGS*.shp')[0]
        else:
            aoi_fn = glob.glob(site_name + '/AOIs/*_outline.shp')[0]
        aoi = gpd.read_file(aoi_fn)
        aoi = aoi.to_crs('EPSG:4326') # reproject to WGS84 for compatibility
        # concatenate to all_aois df
        all_aois = pd.concat([all_aois, aoi])

        # load snowlines
        snowline_fns = glob.glob(site_name + '/imagery/snowlines/*_snowline.csv')
        snowlines_site = gpd.GeoDataFrame()
        for snowline_fn in snowline_fns:
            snowline_df = pd.read_csv(snowline_fn)
            if snowline_df['geometry'][0]!='[]':
                snowline_df['geometry'] = snowline_df['geometry'].apply(wkt.loads)
                snowline_gdf = gpd.GeoDataFrame(snowline_df, geometry='geometry').set_crs(snowline_df['CRS'][0])
                # reproject to WGS84 for compatibility
                snowline_gdf = snowline_gdf.to_crs('EPSG:4326')
                snowline = snowline_gdf
            else:
                snowline = snowline_df
            snowlines_site = pd.concat([snowlines_site, snowline])
        # concatenate to full snowlines df
        all_snowlines = pd.concat([all_snowlines, snowlines_site])

    # reset df indices
    all_aois.reset_index(drop=True, inplace=True)
    all_snowlines.reset_index(drop=True, inplace=True)
    # read datetimes in datetime64[ns] format
    all_snowlines['datetime'] = pd.to_datetime(all_snowlines['datetime'], format='mixed')
    # set O1 and O2 regions datatype to float
    all_aois[['O1Region', 'O2Region']] = all_aois[['O1Region', 'O2Region']].astype(float)
    # rename Benchmark Glacier site_name column as RGIId
    all_snowlines.loc[all_snowlines['site_name']=='Wolverine', 'site_name'] = 'RGI60-01.09162'
    all_snowlines.loc[all_snowlines['site_name']=='Gulkana', 'site_name'] = 'RGI60-01.00570'
    all_snowlines.loc[all_snowlines['site_name']=='LemonCreek', 'site_name'] = 'RGI60-01.01104'
    all_snowlines.loc[all_snowlines['site_name']=='SouthCascade', 'site_name'] = 'RGI60-02.18778'
    all_snowlines.loc[all_snowlines['site_name']=='Sperry', 'site_name'] = 'RGI60-02.17023'

    # save to file
    all_snowlines.to_csv('all_snowlines.csv', index=False)
    all_aois.to_csv('all_aois.csv', index=False)
    print('all_snowlines and all_aois saved to file as CSVs')

In [ ]:
# -----Display unique subregions and number in each
all_aois = all_aois.sort_values(by=['O1Region', 'O2Region']).reset_index(drop=True)
unique_subregion_counts = all_aois[['O1Region', 'O2Region']].value_counts().reset_index(name='count')
unique_subregion_counts = unique_subregion_counts.sort_values(by=['O1Region', 'O2Region']).reset_index(drop=True)
print('Total number of sites with snowlines = ' + str(np.sum(unique_subregion_counts['count'].values)))
# grab unique subregions
unique_subregions = unique_subregion_counts[['O1Region', 'O2Region']].values
unique_subregion_counts

## Plot some figures

In [ ]:
# -----Define function for determining name and color for each subregion
def determine_subregion_name_color(o1, o2):

    if (o1==1.0) and (o2==1.0):
        subregion_name, color = 'Brooks Range', 'c'
    elif (o1==1.0) and (o2==2.0):
        subregion_name, color = 'Alaska Range', '#1f78b4'
    elif (o1==1.0) and (o2==3.0):
        subregion_name, color = 'Aleutians', '#b2df8a'
    elif (o1==1.0) and (o2==4.0):
        subregion_name, color = 'W. Chugach Mtns.', '#33a02c'
    elif (o1==1.0) and (o2==5.0):
        subregion_name, color = 'St. Elias Mtns.', '#fb9a99'
    elif (o1==1.0) and (o2==6.0):
        subregion_name, color = 'N. Coast Ranges', '#e31a1c'
    elif (o1==2.0) and (o2==1.0):
        subregion_name, color = 'N. Rockies', '#cab2d6'
    elif (o1==2.0) and (o2==2.0):
        subregion_name, color = 'N. Cascades', '#fdbf6f'
    elif (o1==2.0) and (o2==3.0):
        subregion_name, color = 'C. Rockies', '#9657d9'
    elif (o1==2.0) and (o2==4.0):
        subregion_name, color = 'S. Cascades', '#ff7f00'
    elif (o1==2.0) and (o2==5.0):
        subregion_name, color = 'S. Rockies', '#6a3d9a'
    else:
        subregion_name = 'O1:' + o1region + ' O2:' + o2region
        color= 'k'
        
    return subregion_name, color

### Plot geographic distribution of all sites

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = plt.subplot(1, 1, 1)
crs = 'EPSG:9822' # Albers Equal Conic projection
# iterate over unique subregions
i=0
for o1region, o2region in unique_subregions:
        
    # grab all_aois in subregion
    all_aois_subregion = all_aois.loc[(all_aois['O1Region']==o1region) & (all_aois['O2Region']==o2region)].reset_index(drop=True)
    # convert to geopandas GeoDataFrame and reproject
    all_aois_subregion_gdf = gpd.GeoDataFrame(all_aois_subregion, geometry='geometry', crs='EPSG:4326')
    all_aois_subregion_gdf = all_aois_subregion_gdf.to_crs(crs)
    # grab RGI IDs in subregion
    ids_subregion = all_aois_subregion['RGIId']
    # grab snowlines with matching names
    snowlines_subregion = all_snowlines[all_snowlines['site_name'].isin(ids_subregion)]
    # determine subregion name and color for plotting
    subregion_name, color = determine_subregion_name_color(o1region, o2region)
    # iterate over AOIs in subregion
    for j in range(0, len(all_aois_subregion)):
        # grab polygon
        polygon = all_aois_subregion_gdf.iloc[j]['geometry']
        # add label on first polygon in region
        if j==0:
            label = subregion_name + ' (N=' + str(len(all_aois_subregion_gdf)) + ')'
        else:
            label='_nolegend_'
        # plot
        ax.plot(*polygon.exterior.xy, color=color, label=label)

    i+=1

# add basemap
ctx.add_basemap(ax=ax, source=ctx.providers.Esri.WorldGrayCanvas, crs=crs, attribution=False)
ax.set_xticks([])
ax.set_yticks([])
ax.legend(loc='upper right', bbox_to_anchor=[1.3, 0.5, 0.2, 0.2])
plt.show()

# save figure
if save_figures:
    fig.savefig(figures_out_path + 'study_sites.png', dpi=300, bbox_inches='tight')
    print('figure saved to file: ' + figures_out_path + 'study_sites.png')

### Plot distributions for site characteristics

In [ ]:
# -----Define parameters to plot
params = ['Area', 'Zmed', 'Slope', 'Aspect', 'TermType', 'Surging']
units = ['[km$^2$]', '[m.a.s.l.]', '[deg]', '[deg]', '', '']
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
# Iterate over params
for i, param in enumerate(params):
    
    # param specific settings
    nbins=10
    if param=='TermType':
        xticks = [0, 1, 2, 3, 4, 5, 9]
        xtick_labels = ['Land-terminating', 'Marine-terminating', 'Lake-terminating', 'Dry calving', 
                        'Regenerated', 'Shelf-terminating', 'Not assigned']
    elif param=='Surging':
        xticks = [0, 1, 2, 3, 9]
        xtick_labels = ['No evidence', 'Possible', 'Probable', 'Observed', 'Not assigned']
    else:
        xticks = np.linspace(np.round(np.nanmin(all_aois[param])), np.round(np.nanmax(all_aois[param])), num=nbins-1)
    
    # plot
    fig = plt.figure(figsize=(8,4))
    counts, edges, bars = plt.hist(all_aois[param], bins=nbins, edgecolor='white', linewidth=1)
    plt.bar_label(bars)
    if (param=='TermType') or (param=='Surging'):
        plt.xticks(xticks, xtick_labels, rotation=45, ha='right')
#     plt.grid()
    plt.title(param)
    plt.xlabel(units[i])
    plt.show()


### Plot all snowlines, with colors distinguishing by subregion

In [ ]:
plt.rcParams.update({'font.size':16})
fig, ax = plt.subplots(1,1,figsize=(16,8))
# iterate over unique subregions
i=0
for o1region, o2region in unique_subregions:
    # grab all_aois in subregion
    all_aois_subregion = all_aois.loc[(all_aois['O1Region']==o1region) & (all_aois['O2Region']==o2region)].reset_index(drop=True)
    # grab RGI IDs in subregion
    ids_subregion = all_aois_subregion['RGIId']
    # grab snowlines with matching names
    snowlines_subregion = all_snowlines[all_snowlines['site_name'].isin(ids_subregion)]
    # determine subregion name and color for plotting
    subregion_name, color = determine_subregion_name_color(o1region, o2region)
    # iterate over AOIs in subregion
    for j in range(0, len(all_aois_subregion)):
        # plot dummy point for legend
        if j==0:
            label = subregion_name + ' (N=' + str(len(all_aois_subregion)) + ')'
            ax.plot(np.datetime64('2000-01-01'),
                    snowlines_subregion['snowline_elevs_median_m'].values[0], 'o',
                    markeredgewidth=3, markeredgecolor=color, markerfacecolor='None',
                    markersize=10, label=label)
        # plot time series
        ax.plot(snowlines_subregion['datetime'],
                snowlines_subregion['snowline_elevs_median_m'], 'o',
                markeredgecolor=color, markerfacecolor='None', markersize=2)

    i+=1

# adjust axes
ax.grid()
ax.set_xlim(np.datetime64('2013-04-01'), np.datetime64('2023-11-01'))
ax.set_ylabel('Median snowline elevation [m]')
ax.legend(loc='center left', bbox_to_anchor=[1.0, 0.4, 0.2, 0.2])
plt.show()

# save figure
if save_figures:
    fig.savefig(figures_out_path + 'snowline_timeseries.png', dpi=300, bbox_inches='tight')
    print('figure saved to file: ' + figures_out_path + 'snowline_timeseries.png')

In [ ]:
# -----Plot snowline timeseries for each subregion separately
# grab unique subregions
unique_subregions = unique_subregion_counts[['O1Region', 'O2Region']].values
# iterate over unique subregions
i=0
for o1region, o2region in unique_subregions:
    # grab all_aois in subregion
    all_aois_subregion = all_aois.loc[(all_aois['O1Region']==o1region) & (all_aois['O2Region']==o2region)].reset_index(drop=True)
    # grab RGI IDs in subregion
    ids_subregion = all_aois_subregion['RGIId']
    # grab snowlines with matching names
    snowlines_subregion = all_snowlines[all_snowlines['site_name'].isin(ids_subregion)]
    # determine subregion name and color for plotting
    subregion_name, color = determine_subregion_name_color(o1region, o2region)
    # iterate over AOIs in subregion
    fig, ax = plt.subplots(1,1,figsize=(12,8))
    # determine title
    title = subregion_name + ' (N=' + str(len(all_aois_subregion)) + ')'
    # plot
    grouped = snowlines_subregion.groupby('site_name')
    cmap = plt.cm.viridis
    j=0
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='datetime', y='snowline_elevs_median_m', label=key, color=cmap(j/len(ids_subregion)))
        j+=1
    ax.grid()
    ax.set_ylabel('Median snowline elevation [m]')
    ax.set_title(title)
    ax.legend(loc='center left', bbox_to_anchor=[1.0, 0.4, 0.2, 0.2])
    plt.show()

    # save figure
    if save_figures:
        fig.savefig(figures_out_path + 'snowline_time_series_' + subregion_name + '.png', dpi=300, bbox_inches='tight')

    i+=1



### Plot snowline timeseries with colors distinguishing terrain parameters

In [ ]:
# aspect
aspect_bins = [[0, 90], [90, 180], [180, 270], [270, 360]]
aspect_names = ['N/NW', 'S/SW', 'S/SE', 'N/NE']
plt.rcParams.update({'font.size':12})
fig, ax = plt.subplots(len(aspect_bins), 1, figsize=(10, 6*len(aspect_bins)))
for i, aspect_bin in enumerate(aspect_bins):
    # select site names in aspect bin
    ids_bin = all_aois.loc[(all_aois['Aspect'] > aspect_bin[0]) & (all_aois['Aspect'] <= aspect_bin[1])]['RGIId']
    # grab snowlines with matching names
    snowlines_bin = all_snowlines[all_snowlines['site_name'].isin(ids_bin)]
    # plot
    snowlines_bin.groupby(by='site_name', group_keys=True).plot(ax=ax[i], x='datetime', 
                                                                y='snowline_elevs_median_m', 
                                                                kind='scatter', legend=False)
    ax[i].grid()
    ax[i].set_title(aspect_names[i] + ' aspect')
    # turn off xlabel
    ax[i].axes.get_xaxis().get_label().set_visible(False)

plt.show()

In [ ]:
# plot each subregion with colors differentiating aspect
import matplotlib
# define colormap for aspect
cmap = plt.cm.twilight_shifted

# iterate over unique subregions
for o1region, o2region in unique_subregions:
    # grab all_aois in subregion
    all_aois_subregion = all_aois.loc[(all_aois['O1Region']==o1region) & (all_aois['O2Region']==o2region)]
    
    # grab RGI IDs in subregion
    ids_subregion = all_aois_subregion['RGIId']
    # set up figure
    fig, ax = plt.subplots(1, 1, figsize=(12,6))
    # iterate over ids
    for ID in ids_subregion.values:
        site_aspect = all_aois.loc[all_aois['RGIId']==ID]['Aspect'].values[0]
        site_snowlines = all_snowlines.loc[all_snowlines['site_name']==ID]
        # plot
        ax.plot(site_snowlines['datetime'], site_snowlines['AAR'],
                'o', markerfacecolor=cmap(site_aspect/360), markeredgecolor='k', markersize=5)
        ax.set_ylabel('Accumulation area ratio')
        ax.set_title(subregion_name)
        ax.grid(True)
        ax.set_xlim(np.datetime64('2013-05-01'), np.datetime64('2022-12-01'))
    # add colorbar
    fig.colorbar(matplotlib.cm.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=0, vmax=360), cmap=cmap),
                 ax=ax, label='Aspect [degrees]', shrink=0.75)
    plt.show()